In [1]:
# Train model
from modelpretrained import NN
from customDataModulepretrained import CustomDataModule
import configpretrained
from callbacks import MyPrintingCallback, EarlyStopping
import pytorch_lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import seed_everything
import torch
import torchvision.models as models

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.



if __name__ == "__main__":
    
    model = models.vgg16(weights='VGG16_Weights.DEFAULT')
    
    logger = TensorBoardLogger("tb_logs", name="my_model") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        model=model,
        input_shape=configpretrained.INPUT_SHAPE,
        num_classes=configpretrained.NUM_CLASSES,
        learning_rate=configpretrained.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModule(
        data_dir=configpretrained.DATA_DIR,
        train_csv=configpretrained.TRAIN_CSV_1,
        val_csv=configpretrained.VAL_CSV_1,
        test_csv=configpretrained.TEST_CSV,
        batch_size=configpretrained.BATCH_SIZE,
        num_workers=configpretrained.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=configpretrained.ACCELERATOR,
        devices=configpretrained.DEVICES,
        min_epochs=configpretrained.MIN_EPOCHS,
        max_epochs=configpretrained.MAX_EPOCHS,
        deterministic=configpretrained.DETERMINISTIC#,
#        callbacks=[MyPrintingCallback(), EarlyStopping(monitor="val_loss")],
    )  # deterministic ensures random seed reproducibility

    trainer.fit(model, dm)  # it will automatically know which dataloader to use

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | model        | VGG                | 138 M 
1 | classifier   | Linear             | 2.0 K 
2 | loss_fn      | CrossEntropyLoss   | 0     
3 | train_acc    | MulticlassAccuracy | 0     
4 | val_acc      | MulticlassAccuracy | 0     
5 | train_recall | MulticlassRecall   | 0     
----------------------------------------------------
2.0 K     Trainable params
138 M     Non-trainable params
138 M     Total params
553.438   Total estimated model params size (MB)


Epoch 0: 100%|██████████| 321/321 [01:58<00:00,  2.71it/s, v_num=22]       
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 321/321 [01:49<00:00,  2.93it/s, v_num=22, train_loss=0.301, train_accuracy=0.919, train_recall=0.919]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 321/321 [01:56<00:00,  2.76it/s, v_num=22, train_loss=0.294, train_accuracy=0.917, train_recall=0.917]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 321/321 [02:35<00:00,  2.07it/s, v_num=22, train_loss=0.283, train_accuracy=0.918, train_recall=0.918]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 321/321 [02:36<00:00,  2.05it/s, v_num=22, train_loss=0.283, train_accuracy=0.918, train_recall=0.918]


In [17]:
model

NN(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): MaxPo

In [18]:
model2 = models.vgg16(weights='VGG16_Weights.DEFAULT')
model2.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [19]:
# returns the size of the output tensor going into the Linear layer from the conv block.
model2.classifier[6].out_features

1000